## Setup

In [ ]:
from dotenv import load_dotenv
import os
import boa
from eth_account import Account

load_dotenv()
PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")
# RPC_URL = "https://sepolia.optimism.io"
RPC_URL = "https://sepolia.base.org"
# RPC_URL = "https://mainnet.optimism.io"
deployer = Account.from_key(PRIVATE_KEY)
eth_env = boa.set_network_env(RPC_URL)
# boa.set_env(eth_env)

# this automatically sets the eoa as the deployer
boa.env.add_account(deployer)
print(f"Deploying with {deployer.address} on Chain id {boa.env.evm.patch.chain_id}")

# I. Deployment

### 1. ExampleMessenger

In [ ]:
# II. Then deploy the DepositLimitModule
contract_deployer = boa.load_partial("../contracts/ExampleMessenger.vy")
LZ_ENDPOINT_BASE_SEPOLIA = "0x6EDCE65403992e310A62460808c4b910D972f10f"
LZ_ENDPOINT_ID = 40245

contract = contract_deployer(
    LZ_ENDPOINT_BASE_SEPOLIA,  # endpoint on sepolia base
    500_000,  # default gas limit
)

print(f"Example Messenger deployed at {contract.address}")

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [ ]:
from web3 import Web3
import os
import logging
import subprocess

LZ_ENDPOINT_ID = 40245
LZ_CHAIN_ID = 84532


def get_vyper_abi():
    command = ["vyper", "../contracts/ExampleMessenger.vy", "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
# RPC endpoints

CONTRACT_ADDRESS = contract.address

try:
    pk = os.environ["WEB3_TESTNET_PK"]
except KeyError:
    raise EnvironmentError("Please set WEB3_TESTNET_PK environment variable")
# execute vyper contracts/LZContract.vy -f abi_python in cli and get the output
ABI = get_vyper_abi()

w3 = Web3(Web3.HTTPProvider("https://sepolia.base.org"))
contract_w3 = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)
account = w3.eth.account.from_key(pk)


def send_tx(func, acc, value=0):
    tx = func.build_transaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "value": value,
        }
    )
    tx["gas"] = int(5 * w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash

## Prepare delegate and config

In [ ]:
# func = contract_w3.functions.set_lz_delegate(account.address)
# tx_hash = send_tx(func, account)
# logging.info(f"Added self as delegate: {tx_hash.hex()}")

SEND_LIB = "0xC1868e054425D378095A003EcbA3823a5D0135C9"
RECEIVE_LIB = "0x12523de19dc41c91F7d2093E0CFbB76b17012C8d"
READ_LIB = "0x29270F0CFC54432181C853Cd25E2Fb60A68E03f2"

# lib addresses for base sepolia
# send lib
func = contract_w3.functions.set_lz_send_lib(LZ_ENDPOINT_ID, SEND_LIB)
tx_hash1 = send_tx(func, account)
# # receive lib
# func = contract_w3.functions.set_lz_receive_lib(LZ_ENDPOINT_ID, RECEIVE_LIB)
# tx_hash2 = send_tx(func, account)
# # read lib
# func = contract_w3.functions.set_lz_send_lib(contract_w3.functions.LZ_READ_CHANNEL().call(), READ_LIB)
# tx_hash3 = send_tx(func, account)
# for tx in [tx_hash1, tx_hash2, tx_hash3]:
#     print(f"Added lib: {tx.hex()}")

## 1. LZ Send & Receive

In [ ]:
# add self as peer

func = contract_w3.functions.set_peer(LZ_ENDPOINT_ID, CONTRACT_ADDRESS)
tx_hash = send_tx(func, account)
logging.info(f"Added self as peer tx: {tx_hash.hex()}")

In [ ]:
gas_limit = 500_000
msg = "170475436437825620930817601234267694881687829390282260281137596999800372275961"

fee = contract_w3.functions.quote_message_fee(
    LZ_ENDPOINT_ID, CONTRACT_ADDRESS, msg, _gas_limit=gas_limit
).call()
logging.info(f"Fee: {fee}")

func = contract_w3.functions.send_message(
    _dst_eid=LZ_ENDPOINT_ID, _receiver=CONTRACT_ADDRESS, _message=msg, _gas_limit=gas_limit
)
tx_hash = send_tx(func, account, 3 * fee)
logging.info(f"Tx: {tx_hash.hex()}")
# after we sent the message we can check it on testnet.layerzeroscan.com (use address or txid)
# shortly after, it should be pinged back (we send to ourselves, and event be fired)

## 2. LzRead

In [ ]:
# set self as read peer
# LZ_READ_CHANNEL = 4294967294
LZ_READ_CHANNEL = contract_w3.functions.LZ_READ_CHANNEL().call()
func = contract_w3.functions.set_peer(LZ_READ_CHANNEL, CONTRACT_ADDRESS)
tx_hash = send_tx(func, account)
logging.info(f"Set peer tx: {tx_hash.hex()}")

In [ ]:
# prepare reading calldata
from vyper.utils import method_id

method_str = "dummy_endpoint(uint256)"
num = 12345
calldata = method_id(method_str) + boa.util.abi.abi_encode("(uint256)", (num,))

# read the contract itself
res = contract_w3.functions.dummy_endpoint(num).call()

logging.info(f"Read: {res}")

# now quote lzread fee
fee = contract_w3.functions.quote_read_fee(
    LZ_ENDPOINT_ID, CONTRACT_ADDRESS, calldata, _gas_limit=gas_limit
).call()
logging.info(f"Fee: {fee}")


# now request read
func = contract_w3.functions.request_read(
    LZ_ENDPOINT_ID, CONTRACT_ADDRESS, calldata, _gas_limit=gas_limit
)

tx = func.build_transaction(
    {
        "from": account.address,
        "nonce": w3.eth.get_transaction_count(account.address),
        "value": int(fee * 3),
    }
)
tx["gas"] = int(10 * w3.eth.estimate_gas(tx))
signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
logging.info(f"Tx: {tx_hash.hex()}")

In [ ]:
# test calldata
import requests

print(calldata.hex())

rpc_endpoint = "https://sepolia.base.org"
# perfor eth_call request to contract using calldata
r = requests.post(
    rpc_endpoint,
    json={
        "jsonrpc": "2.0",
        "method": "eth_call",
        "params": [{"to": CONTRACT_ADDRESS, "data": "0x" + calldata.hex()}, "latest"],
        "id": 1,
    },
)
response = r.json()["result"]
# print(r.json()['result'])
print(response)
contract_code = """
@external
@view
def message_as_string(_message: Bytes[128]) -> String[128]:
    message: String[128] = convert(_message, String[128])
    return message
"""
with boa.swap_env(boa.Env()):
    tmp_contract = boa.loads(contract_code)

    res = tmp_contract.message_as_string(bytes.fromhex(response[2:]))
    print(res)